<a href="https://colab.research.google.com/github/podo1skii/spbu_cldocs/blob/master/Task_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [4]:
lines = list(open('./news_train.txt', 'r', encoding='utf-8'))

In [5]:
data = []
category_data = []

In [6]:
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stemmer = SnowballStemmer("russian")

def get_stem(line):
  result = []
  line = line.replace('\t', ' ')
  for word in line.split(' '):
    result.append(stemmer.stem(word))
  return result

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
for line in lines:
  i = line.index('\t')
  category = line[0:i]
  article = line[i+1:]
  data.append(get_stem(article))
  category_data.append(category)

In [8]:
len(lines)

15000

In [9]:
from gensim.models import Word2Vec

model = Word2Vec()
model.build_vocab(data)

In [10]:
model.train(data, total_examples=model.corpus_count, epochs=20, report_delay=1)

(45741087, 59292180)

In [11]:
model.wv.most_similar(positive=['спорт'], topn=5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('спорта,', 0.6666858792304993),
 ('спорта.', 0.6321720480918884),
 ('культур', 0.599403977394104),
 ('хокке', 0.5543338060379028),
 ('фигурн', 0.5477985739707947)]

In [12]:
model.wv.most_similar(positive=['муж'], topn=5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('жен', 0.8240318298339844),
 ('супруг', 0.8178457021713257),
 ('подруг', 0.7589892148971558),
 ('отц', 0.7466211915016174),
 ('мужа,', 0.7349746227264404)]

In [13]:
len(category_data)

15000

In [28]:
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np

In [15]:
len(lines)

15000

In [16]:
sentences = []
for line in lines:
  i = line.index('\t')
  article = line[i+1:]
  sentences.append(article)

In [17]:
len(sentences)

15000

In [18]:
stop_words=stopwords.words('russian')
vectorizer = TfidfVectorizer(max_features=5000, stop_words=stop_words)

X = vectorizer.fit_transform(sentences)
print(X.shape)

(15000, 5000)


In [19]:
svm_classifier = svm.SVC()
svm_classifier.fit(X, category_data)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [20]:
lines = list(open('./news_test.txt', 'r', encoding='utf-8'))
test_sentences = []
test_category = []
for line in lines:
  first_index = line.index('\t')
  last_index = line.rindex('\t')
  article = line[last_index+1:]
  test_category.append(line[0:first_index])
  test_sentences.append(article)


X_test = vectorizer.transform(test_sentences)

In [21]:
len(test_sentences)

3000

In [22]:
X_test.shape

(3000, 5000)

In [23]:
np.array(test_category).shape

(3000,)

In [24]:
Y_test_predicted = svm_classifier.predict(X_test)

In [25]:
Y_test_predicted.shape

(3000,)

In [26]:
result = accuracy_score(np.array(test_category), Y_test_predicted)

In [27]:
result

0.8696666666666667

In [32]:
log_reg = LogisticRegression()

log_reg.fit(X, category_data)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
Y_log_predicted = log_reg.predict(X_test)

In [35]:
result_log = accuracy_score(np.array(test_category), Y_log_predicted)

In [36]:
result_log

0.8716666666666667